<a href="https://colab.research.google.com/github/rafaelquesadaal/03MIAR---Algoritmos-de-Optimizacion/blob/main/TrabajoPractico/Trabajo_Pr%C3%A1ctico_Rafael_Quesada_Alpizar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Rafael Quesada Alpízar  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/1oSXK4tJnS-ihaKjRklv5Mq8zseR61OQ4?usp=sharing
<br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema:(copiar enunciado)

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

Número de actores: 10
Número de tomas : 30
Actores/Tomas : https://bit.ly/36D8IuK
- 1 indica que el actor participa en la toma
- 0 en caso contrario






                                        

#Modelo
- ¿Como represento el espacio de soluciones?

En este caso, el espacio de soluciones está representado por todas las posibles asignaciones que se pueden hacer para cada día de rodamiento, esto tomando como restricción las reglas previamente definidas. Para este algoritmo se utiliza una representación matricial, donde una matriz contiene las soluciones encontradas por el algoritmo; cada fila de la matriz es una solución, que a su vez es una matriz donde cada fila representa un día de rodaje, la cual contiene la lista de tomas por hacer. De esta forma el espacio de soluciones está dado en una matriz de 3 dimensiones.

- ¿Cual es la función objetivo?

La función objetivo de este problema es minimizar el coste total para las sesiones de doblaje, esto tomando en cuenta que el gasto generado por los actores depende de ir o no un día al estudio, y no precisamente de las tomas ejecutadas, es decir, podemos asumir que el coste por día es la suma de los actores presentes ese día (según las tomas correspondientes). Esto es lo que se desea minimizar y tomando en cuenta las restricciones previamente definidas, por ejemplo que no se puedan grabar más de 6 tomas por día.

- ¿Como implemento las restricciones?

Las restricciones en este caso, son implementadas en la forma de calcular las soluciones iniciales, los cruces y mutaciones. Por ejemplo, las soluciones iniciales siempre contienen 6 tomas por día de grabación y luego, para los cruces, solamente se intercambian posiciones entre padres, sin añadir más o menos elementos; y para las mutaciones solamente se muta una toma en específico, de tal forma que las longitudes siempre van a respetar la condición de no más de 6 tomas diarias.


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

El orden de complejidad es es *O(n!)* y no es posible contabilizar de manera precisa el espacio de soluciones posible, aún considerando las restricciones y condiciones de entrada.


#Diseño
- ¿Que técnica utilizo? ¿Por qué?



Para este problema se decidió utilizar un algoritmo genético, ya que este tipo de técnica es especialmente bueno para espacios de soluciones grandes y complejos. Al ser un algoritmo heurístico nos permite encontrar soluciones óptimas o cercanas a ellas, además es una técnica sencilla de entender e implementar que trae buenos resultados.

Como complemento algunas características del algoritmo genético utilizado es que utiliza ambos tipos de operadores genéticos, el cruce para mayor intensificación y la mutación para más diversificación, de esta forma se tiene un buen balance. Para el caso del cruce se utilizó la técnica regular de 2 padres con cruce en 1 punto (1-point crossover).

Para la población inicial se utilizaron solamente 2 individuos, uno generado de forma ordenada, es decir las primeras 6 tomas el primer día, las siguientes 6 al día siguiente y así sucesivamente, y el otro individuo fue generado al azar. Se utilizaron 2 solamente, para poder empezar a cruzarlos entre sí. Además, estas 2 soluciones iniciales se crearon con una estructura de exactamente 6 tomas por día, esto para hacer de nuestro espacio de soluciones, solamente individuos con esa estructura, de esta forma se garantiza que en base a nuestra técnica, las soluciones resultantes tengan la misma distribución y recortemos la exploración de soluciones con más días y menos tomas por día.

Un aspecto importante es que las soluciones se fueron descartando en base a un elitismo, donde se dejaban a los mejores individuos actuales en el espacio de soluciones basados en el porcentaje de etilismo suministrado.

Como último aspecto relevante en esta solución es los resultados generados por la mutación, ya que en las etapas tempranas del desarrollo del algoritmo no se estaba utilizando la mutación y el mejor resultado obtenido no bajaba de un coste de 34. Sin embargo, cuando se incluyó la mutación se pudo ver una mejoría en las soluciones encontradas, ya que se ha alcanzado soluciones encontradas de hasta 28 de coste. Esto es muy importante recalcar ya que como se vio en clase, esta técnica nos puede ayudar a salir de mínimos locales, lo cual parece que era lo que se estaba encontrando al principio.

#Algoritmo

In [42]:
import numpy as np
import random

# Cargamos las tomas, se tomó desde un archivo CSV para mayor facilidad. (Se convirtió previamente el original)
datos_tomas = [
    [1,1,1,1,1,0,0,0,0,0],
    [0,0,1,1,1,0,0,0,0,0],
    [0,1,0,0,1,0,1,0,0,0],
    [1,1,0,0,0,0,1,1,0,0],
    [0,1,0,1,0,0,0,1,0,0],
    [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,1,0],
    [1,1,1,0,1,0,0,1,0,0],
    [1,1,1,1,0,1,0,0,0,0],
    [1,0,0,1,1,0,0,0,0,0],
    [1,0,1,0,0,1,0,0,0,0],
    [1,1,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,1],
    [1,0,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,1,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0],
    [1,0,1,1,1,0,0,0,0,0],
    [0,0,0,0,0,1,0,1,0,0],
    [1,1,1,1,0,0,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,1],
    [1,0,1,0,1,0,0,0,1,0],
    [0,0,0,1,1,0,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0],
    [1,0,0,0,1,1,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0]
]


matriz_tomas = np.array(datos_tomas)

# Genera 1 solución inicial
def solucion_inicial(tomas_por_dia, solucion_random=False):
    mejor_costo = 99999
    mejor_planificacion = None
    tomas_disponibles = list(range(len(tomas_por_dia)))
    if solucion_random:
      random.shuffle(tomas_disponibles)

    dias = []
    while tomas_disponibles:
        if len(tomas_disponibles) >= 6:
            dia_actual = tomas_disponibles[:6]
            tomas_disponibles = tomas_disponibles[6:]
            dias.append(dia_actual)
        else:
            break  # No consideraremos días con menos de 6 tomas
    costo_actual = sum(len(set(actor for toma_id in dia for actor, presencia in enumerate(tomas_por_dia[toma_id]) if presencia == 1)) for dia in dias)
    if costo_actual < mejor_costo:
        mejor_costo = costo_actual
        mejor_planificacion = dias
    return mejor_costo, mejor_planificacion

# Este método calcula el coste por día
def coste_toma_dia(dia):
  coste_total_dia = 0
  for i in range(10): # 10 actores
    for j in range(len(dia)):
      if matriz_tomas[dia[j]][i] == 1:
        coste_total_dia += 1
        break
  return coste_total_dia

def coste_total(solucion):
  coste_total_solucion = 0
  for i in range(len(solucion)):
    coste_total_solucion += coste_toma_dia(solucion[i])
  return coste_total_solucion

# Método que evalúa todas las soluciones y devuelve la mejor y el mejor coste.
def evaluacion_soluciones(soluciones):
  mejor_solucion = []
  mejor_coste = 9999999
  # print("Evaluando " + str(len(soluciones)) + " soluciones")
  for solucion in soluciones:
    coste = coste_total(solucion)
    if coste < mejor_coste:
      mejor_coste = coste
      mejor_solucion = solucion
  return mejor_solucion, mejor_coste

# Método que cruza 2 padres seleccionados al azar dentro del espacio de soluciones actual
def cruzar(soluciones, mutacion, cruce_adicional=0.5):
  padre1_random = random.randint(0, len(soluciones)-1)
  padre2_random = random.choice([i for i in range(len(soluciones)) if i not in [padre1_random]])
  # Se aplanan las matrices para hacer el cruce mas comodamente, luego se vuelven a su dimensionalidad original
  solucion_padre1_flatten = soluciones[padre1_random].flatten()
  solucion_padre2_flatten = soluciones[padre2_random].flatten()
  pivote_cruce = 15 # Pivote para partir por la mitad a los padres (Acá se hace el 1-point crossover)
  hijo_flatten = np.concatenate((solucion_padre1_flatten[:pivote_cruce], solucion_padre2_flatten[pivote_cruce:]), axis=0)
  # Se corrige al hijo en caso de que tenga alguna toma repetida
  hijo_flatten = correccion_cruce(hijo_flatten)

  # Se muta al hijo en base a la probabilidad de mutación
  hijo_flatten = mutar_solucion(hijo_flatten, mutacion)
  hijo = np.reshape(hijo_flatten, soluciones[padre1_random].shape)

  hijo = cruce_adicional_solucion(hijo, cruce_adicional)
  return hijo

# Método que hace correcciones a los hijos, en concreto corrige posibles tomas duplicadas
def correccion_cruce(solucion_hijo):
  hijo_limpio, idx = np.unique(solucion_hijo, return_index=True)
  copia_hijo_solucion = solucion_hijo.copy()
  indices_duplicados = np.delete(np.arange(solucion_hijo.size), idx)
  for i in indices_duplicados:
      nueva_toma = random.choice([j for j in range(30) if j not in hijo_limpio])
      copia_hijo_solucion[i] = nueva_toma
      hijo_limpio = np.append(hijo_limpio, nueva_toma)
  return copia_hijo_solucion

# Método para el fitness, donde se seleccionan los mejores elementos del espacio de soluciones basado en el porcentaje de etilismo
def seleccion_soluciones(soluciones, poblacion_minima, elitismo):
  fitness = []
  for solucion in soluciones: # Se guardan todas las soluciones con su respectivo coste
    coste = coste_total(solucion)
    fitness.append((coste, solucion))
  fitness_ordenado = sorted(fitness, key=lambda tup: tup[0])
  if len(fitness_ordenado) >= poblacion_minima: # Solamente se aplica el etilismo cuando tenemos más de 12 soluciones
    indice_corte_elitismo = int(len(fitness_ordenado) * elitismo)
    fitness_ordenado = fitness_ordenado[:indice_corte_elitismo]
  soluciones_seleccionadas = [tup[1] for tup in fitness_ordenado]
  return soluciones_seleccionadas

# Método para mutar una solución, normalmente un hijo recién creado
def mutar_solucion(solucion, mutacion):
  porcentaje_mutacion = random.randint(0, 100)
  if porcentaje_mutacion < mutacion * 100:
    toma_mutacion_index = random.randint(0, len(solucion)-1)

    toma_a_mutar = solucion[toma_mutacion_index]
    nueva_toma_mutada = random.choice([i for i in range(30) if i not in [toma_a_mutar]]) # Hasta el 29 porque son 30 tomas de las que se puede elegir, menos la toma anterior
    toma_mutada_repetida_index = np.where(solucion == nueva_toma_mutada)[0][0] # Indice de la toma nueva mutada, ya que como esta es una solucion completa, ya estaba tomada en otro espacio, entonces se intercambia por el valor anterior

    solucion_previa = solucion.copy()
    solucion[toma_mutacion_index] = nueva_toma_mutada
    solucion[toma_mutada_repetida_index] = toma_a_mutar
    # print("[MUTACION] Se mutó la toma " + str(toma_a_mutar) + " (Index: " + str(toma_mutacion_index) + ") por la toma " + str(nueva_toma_mutada)
    #   + " y se reasignó el valor anterior en " + "(Index: " + str(toma_mutada_repetida_index) + ")")
    # print("Previa: ", solucion_previa)
    # print("Mutada: ", solucion)
  return solucion

# Método para generar un cruce adicional y ser N-point crossover. Este solamente cruza 1 elemento entre padres.
def cruce_adicional_solucion(solucion, cruce_adicional):
  porcentaje_mutacion = random.randint(0, 100)
  if porcentaje_mutacion < cruce_adicional * 100:
    dia_mutacion_1 = random.randint(0, len(solucion)-1)
    toma_mutacion_index_1 = random.randint(0, len(solucion[dia_mutacion_1])-1)
    dia_mutacion_2 = random.randint(0, len(solucion)-1)
    toma_mutacion_index_2 = random.randint(0, len(solucion[dia_mutacion_2])-1)

    toma_temp = solucion[dia_mutacion_1][toma_mutacion_index_1]
    solucion[dia_mutacion_1][toma_mutacion_index_1] = solucion[dia_mutacion_2][toma_mutacion_index_2]
    solucion[dia_mutacion_2][toma_mutacion_index_2] = toma_temp
    # print("[N-POINT CROSSOVER] Se cruzó la toma " + str(toma_mutacion_index_1) + " (" + str(solucion[dia_mutacion_2][toma_mutacion_index_2]) + ")" + " del día " + str(dia_mutacion_1)
    #   + " por la toma " + str(toma_mutacion_index_2) + " (" + str(solucion[dia_mutacion_1][toma_mutacion_index_1]) + ")" + " del día " + str(dia_mutacion_2))
  return solucion


def algoritmo_genetico(poblacion_minima=40, mutacion=.15, elitismo=.5, cruce_adicional=.5, generaciones=100):
  soluciones = []
  _, solucion_inicial_1 = solucion_inicial(matriz_tomas, False) # Primera solución inicial de forma ordenada
  _, solucion_inicial_2 = solucion_inicial(matriz_tomas, True) # Segunda solución inicial de manera aleatoria
  soluciones.append(solucion_inicial_1)
  soluciones.append(solucion_inicial_2)
  soluciones = np.array(soluciones, dtype=object)

  (mejor_solucion, mejor_distancia) = evaluacion_soluciones(soluciones)
  parar = False
  n=0
  while(parar == False):
    # Cruce de soluciones
    nueva_solucion = cruzar(soluciones, mutacion, cruce_adicional)
    nueva_solucion = nueva_solucion[np.newaxis, :]
    soluciones = np.vstack((soluciones, nueva_solucion))
    soluciones = seleccion_soluciones(soluciones, poblacion_minima, elitismo)

    # Evaluación de las mejores soluciones
    (mejor_solucion, mejor_coste) = evaluacion_soluciones(soluciones)
    if n%500 == 0:
      print("\n\t\t**********************************************************************************")
      print("\t\t*    [Generacion #", n, "]    La mejor solución actual tiene un coste de: " , mejor_coste, "    *")
      print("\t\t**********************************************************************************\n")
    if n==generaciones:
      parar = True
    n += 1

  solucion_final = mejor_solucion.copy()
  solucion_final = solucion_final
  # print("Solucion final: \n", mejor_solucion, "\nCoste: ", mejor_coste)



  print("\n\t\t**********************************************************************************")
  print("\t\t*                                SOLUCION FINAL                                  *")
  print("\t\t*                                                                                *")
  for i in range(len(solucion_final)):
    print("\t\t*    Día: ", i+1, "                                                                    *")
    texto = str(solucion_final[i] + 1)
    texto = texto + (" " * (63 - len(texto)))
    print("\t\t*        Tomas: ", texto, "*")
  print("\t\t*                                                                                *")
  print("\t\t*                                                                                *")
  print("\t\t*    --> Coste total: ", mejor_coste, "                                                       *")
  print("\t\t*                                                                                *")
  print("\t\t**********************************************************************************\n")
  return mejor_solucion
82
solucion_definitiva = algoritmo_genetico(poblacion_minima=12, mutacion=.6, elitismo=.4, cruce_adicional=.2, generaciones=3000)



		**********************************************************************************
		*    [Generacion # 0 ]    La mejor solución actual tiene un coste de:  37     *
		**********************************************************************************


		**********************************************************************************
		*    [Generacion # 500 ]    La mejor solución actual tiene un coste de:  31     *
		**********************************************************************************


		**********************************************************************************
		*    [Generacion # 1000 ]    La mejor solución actual tiene un coste de:  30     *
		**********************************************************************************


		**********************************************************************************
		*    [Generacion # 1500 ]    La mejor solución actual tiene un coste de:  29     *
		************************************************************